In [1]:
import os
from urllib.request import urlretrieve
import zipfile

In [2]:
# Dowload population data
data_url = "https://www.abs.gov.au/statistics/people/population/regional-population/2022-23/32180_ERP_2023_SA2_GDA2020.zip"
urlretrieve(data_url, "../data/landing/population.zip")

('../data/landing/population.zip', <http.client.HTTPMessage at 0x1138a6250>)

In [3]:
# Extract the contents of the zip file
with zipfile.ZipFile("../data/landing/population.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/population/")

In [4]:
import geopandas as gpd

# Load the GeoPackage file
gdf = gpd.read_file("../data/population/32180_ERP_2023_SA2_GDA2020.gpkg")

# Check the first few rows of the dataset to understand its structure
print(gdf.head(1))


   State_code_2021  State_name_2021  SA2_code_2021 SA2_name_2021  \
0                1  New South Wales      101021007     Braidwood   

   SA3_code_2021 SA3_name_2021  SA4_code_2021   SA4_name_2021 GCCSA_code_2021  \
0          10102    Queanbeyan            101  Capital Region           1RNSW   

  GCCSA_name_2021  ...  Births_2022_23  Deaths_2022_23  \
0     Rest of NSW  ...              44              41   

   Natural_increase_2022_23  Internal_arrivals_2022_23  \
0                         3                        316   

   Internal_departures_2022_23  Net_internal_migration_2022_23  \
0                          301                              15   

   Overseas_arrivals_2022_23  Overseas_departures_2022_23  \
0                         19                            7   

   Net_overseas_migration_2022_23  \
0                              12   

                                            geometry  
0  MULTIPOLYGON (((149.58424 -35.44426, 149.58432...  

[1 rows x 56 columns]


In [5]:
gdf = gdf[gdf["State_name_2021"] == "Victoria"]
gdf["SA3_name_2021"].unique()

array(['Ballarat', 'Creswick - Daylesford - Ballan',
       'Maryborough - Pyrenees', 'Bendigo',
       'Heathcote - Castlemaine - Kyneton', 'Loddon - Elmore',
       'Barwon - West', 'Geelong', 'Surf Coast - Bellarine Peninsula',
       'Upper Goulburn Valley', 'Wangaratta - Benalla',
       'Wodonga - Alpine', 'Baw Baw', 'Gippsland - East',
       'Gippsland - South West', 'Latrobe Valley', 'Wellington',
       'Brunswick - Coburg', 'Darebin - South', 'Essendon',
       'Melbourne City', 'Port Phillip', 'Stonnington - West', 'Yarra',
       'Boroondara', 'Manningham - West', 'Whitehorse - West', 'Bayside',
       'Glen Eira', 'Kingston', 'Stonnington - East', 'Banyule',
       'Darebin - North', 'Nillumbik - Kinglake', 'Whittlesea - Wallan',
       'Keilor', 'Macedon Ranges', 'Moreland - North', 'Sunbury',
       'Tullamarine - Broadmeadows', 'Knox', 'Manningham - East',
       'Maroondah', 'Whitehorse - East', 'Yarra Ranges', 'Cardinia',
       'Casey - North', 'Casey - South', 'Dan

In [6]:
gdf.columns

Index(['State_code_2021', 'State_name_2021', 'SA2_code_2021', 'SA2_name_2021',
       'SA3_code_2021', 'SA3_name_2021', 'SA4_code_2021', 'SA4_name_2021',
       'GCCSA_code_2021', 'GCCSA_name_2021', 'ERP_2001', 'ERP_2002',
       'ERP_2003', 'ERP_2004', 'ERP_2005', 'ERP_2006', 'ERP_2007', 'ERP_2008',
       'ERP_2009', 'ERP_2010', 'ERP_2011', 'ERP_2012', 'ERP_2013', 'ERP_2014',
       'ERP_2015', 'ERP_2016', 'ERP_2017', 'ERP_2018', 'ERP_2019', 'ERP_2020',
       'ERP_2021', 'ERP_2022', 'ERP_2023', 'ERP_change_number_2022_23',
       'ERP_change_per_cent_2022_23', 'Area_km2',
       'Pop_density_2023_people_per_km2', 'Births_2021_22', 'Deaths_2021_22',
       'Natural_increase_2021_22', 'Internal_arrivals_2021_22',
       'Internal_departures_2021_22', 'Net_internal_migration_2021_22',
       'Overseas_arrivals_2021_22', 'Overseas_departures_2021_22',
       'Net_overseas_migration_2021_22', 'Births_2022_23', 'Deaths_2022_23',
       'Natural_increase_2022_23', 'Internal_arrivals_2022_2

In [7]:
import folium
import branca.colormap as cm

In [8]:
# Create directories to store plots
os.makedirs("../plots/migrants/", exist_ok=True)

In [9]:
# Create a map centered on Victoria
m = folium.Map(location=[-37.4713, 144.7852], zoom_start=7)

# Create a color map using branca
colormap = cm.LinearColormap(
    colors=['yellow', 'orange', 'red'],
    vmin=gdf['Net_overseas_migration_2021_22'].min(),
    vmax=gdf['Net_overseas_migration_2021_22'].max(),
    caption='Net_overseas_migration_2021_22'
)

# Define a function to style the features
def style_function(feature):
    return {
        'fillOpacity': 0.7,
        'weight': 0.5,
        'fillColor': colormap(feature['properties']['Net_overseas_migration_2021_22']),
        'color': 'black'
    }

# Add the GeoDataFrame to the map with Folium
folium.GeoJson(
    gdf,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['SA3_name_2021', 'Net_overseas_migration_2021_22'], 
        aliases=['Suburb Name', 'Net overseas migration 21-22']
    )
).add_to(m)

# Add the colormap legend to the map
colormap.add_to(m)

# Save and display the map
m.save("../plots/migrants/suburbs_net_overseas_migration_map_21_22.html")

In [10]:
# Create a map centered on Victoria
m = folium.Map(location=[-37.4713, 144.7852], zoom_start=7)

# Create a color map using branca
colormap = cm.LinearColormap(
    colors=['yellow', 'orange', 'red'],
    vmin=gdf['Net_overseas_migration_2022_23'].min(),
    vmax=gdf['Net_overseas_migration_2022_23'].max(),
    caption='Net_overseas_migration_2022_23'
)

# Add the GeoDataFrame to the map with Folium
folium.GeoJson(
    gdf,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['SA3_name_2021', 'Net_overseas_migration_2022_23'], 
        aliases=['Suburb Name', 'Net overseas migration 22-23']
    )
).add_to(m)

# Add the colormap legend to the map
colormap.add_to(m)

# Save and display the map
m.save("../plots/migrants/suburbs_net_overseas_migration_map_22_23.html")